In [1]:
import pandas as pd
from datetime import datetime

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

In [3]:
query = """SELECT b.datecreated, b.timestamp, b.biblionumber, bi.itemtype, b.title, b.author, bi.isbn
FROM koha_prod.biblio b
LEFT JOIN koha_prod.biblioitems bi ON bi.biblionumber = b.biblionumber
LEFT JOIN koha_prod.items i ON i.biblionumber = b.biblionumber
WHERE i.itemnumber IS NULL
ORDER BY datecreated"""

In [4]:
items = pd.read_sql(query,db_conn)

In [5]:
items

,datecreated,timestamp,biblionumber,itemtype,title,author,isbn
0,2014-06-30,2022-04-11 10:24:17,213504,LI,L' hygiène des villes à l'exposition de Roubai...,None,None
1,2014-06-30,2022-04-11 10:09:55,84736,DV,Fables,"Jean-Luc Ponty, vl, synth., perc.",None
2,2014-06-30,2014-06-30 17:47:22,151552,LI,Déserts,None,None
3,2014-06-30,2022-04-11 10:17:22,151808,LI,"Autres médecines, autres mœurs",dir. par Françoise Bouchayer,2-86260-177-2
4,2014-06-30,2015-05-18 09:32:01,152576,LI,Le plat pays en voit de toutes les couleurs,texte Alain Chaignon,None
...,...,...,...,...,...,...,...
9574,2022-12-14,2022-12-14 12:06:24,352881,LI,Troublante rencontre,K. D. Richards | Jane Godman,978-2-280-47106-0
9575,2022-12-14,2022-12-14 12:06:24,352882,LI,La maison du mystère,Nora Roberts,978-2-280-45998-3
9576,2022-12-14,2022-12-14 12:06:24,352883,LI,Les embruns du coeur,LaVyrle Spencer,978-2-290-23998-8
9577,2022-12-14,2022-12-14 12:06:24,352884,LI,Espionne,Danielle Steel,978-2-258-19187-7


In [6]:
items.columns

Index(['datecreated', 'timestamp', 'biblionumber', 'itemtype', 'title',
       'author', 'isbn'],
      dtype='object')

In [7]:
items = items.rename(columns={"datecreated":"date de création",
                     "timestamp":"dernier changement le",
                     "itemtype":"type de support",
                     "title":"titre",
                     "author":"auteur"})

In [8]:
items.columns

Index(['date de création', 'dernier changement le', 'biblionumber',
       'type de support', 'titre', 'auteur', 'isbn'],
      dtype='object')

In [17]:
colonnes_a_exporter = ["date de création",
                       "dernier changement le",
                       "biblionumber",
                       "type de support", 
                       "titre", 'auteur', "isbn"]

In [9]:
deadline_2_mois = pd.to_datetime("today") - pd.Timedelta(365/6, unit='D')

In [10]:
anomalies_notice_bib = items[items["dernier changement le"] < deadline_2_mois]

In [11]:
anomalies_notice_bib

,date de création,dernier changement le,biblionumber,type de support,titre,auteur,isbn
0,2014-06-30,2022-04-11 10:24:17,213504,LI,L' hygiène des villes à l'exposition de Roubai...,None,None
1,2014-06-30,2022-04-11 10:09:55,84736,DV,Fables,"Jean-Luc Ponty, vl, synth., perc.",None
2,2014-06-30,2014-06-30 17:47:22,151552,LI,Déserts,None,None
3,2014-06-30,2022-04-11 10:17:22,151808,LI,"Autres médecines, autres mœurs",dir. par Françoise Bouchayer,2-86260-177-2
4,2014-06-30,2015-05-18 09:32:01,152576,LI,Le plat pays en voit de toutes les couleurs,texte Alain Chaignon,None
...,...,...,...,...,...,...,...
8972,2022-10-13,2022-10-14 02:00:16,350683,VD,Compétition officielle,"Mariano Cohn, réal.",None
8973,2022-10-13,2022-10-14 02:00:16,350690,VD,Entre la vie et la mort,"Giordano Gederlini, réal.",None
8974,2022-10-13,2022-10-14 02:00:16,350691,VD,Euphoria - Saisons 1 et 2,"Sam Levinson, réal.",None
8975,2022-10-13,2022-10-14 02:00:15,350701,VD,Ouistreham,"Emmanuel Carrère, réal.",None


In [18]:
anomalies_notice_bib[colonnes_a_exporter].to_excel("../data/liste_anomalies_notices_bib.xlsx",index=False)